In [1]:
import osmnx as ox 
ox.config(use_cache=True, log_console=True)
ox.__version__

'0.11dev'

In [2]:
place = {'city':'San Francisco', 'state':'California', 'country':'USA'}
G = ox.graph_from_place(place, network_type='drive', simplify=True, retain_all=False, 
                        truncate_by_edge=True, clean_periphery=True)

In [3]:
nodes, edges = ox.graph_to_gdfs(G)
edges['highway'].value_counts()

residential                    17884
tertiary                        3540
secondary                       2346
primary                         1783
unclassified                     534
trunk                            229
motorway_link                    140
motorway                          96
primary_link                      83
secondary_link                    74
tertiary_link                     35
[residential, unclassified]       23
living_street                     18
trunk_link                        18
[residential, tertiary]            4
[trunk, motorway]                  4
[primary_link, primary]            1
Name: highway, dtype: int64

## filter out motorway edges

In [4]:
e = [(u, v, k) for u, v, k, d in G.edges(keys=True, data=True) if 'motorway' in d['highway']]

In [5]:
# remove motorway edges then retain only the largest strongly connected component
G.remove_edges_from(e)
G = ox.get_largest_component(G, strongly=True)

In [6]:
nodes, edges = ox.graph_to_gdfs(G)
edges['highway'].value_counts()

residential                    17643
tertiary                        3422
secondary                       2346
primary                         1783
unclassified                     489
trunk                            226
primary_link                      83
secondary_link                    72
tertiary_link                     35
[residential, unclassified]       21
trunk_link                        16
living_street                     11
[residential, tertiary]            4
[primary_link, primary]            1
Name: highway, dtype: int64

## save graphs to disk

save directed graph to disk

In [7]:
ox.save_graphml(G, filename='data/network/sf-directed-no-fwy.graphml', folder='.')

In [8]:
nodes = ox.graph_to_gdfs(G, edges=False, node_geometry=False)
nodes.to_csv('data/network/sf-directed-no-fwy-nodelist.csv', index=True, encoding='utf-8')

save undirected representation of graph to disk

In [9]:
G_undir = ox.get_undirected(G)

In [10]:
ox.save_graphml(G_undir, filename='data/network/sf-undirected-no-fwy.graphml', folder='.')

In [11]:
nodes = ox.graph_to_gdfs(G_undir, edges=False, node_geometry=False)
nodes.to_csv('data/network/sf-undirected-no-fwy-nodelist.csv', index=True, encoding='utf-8')